<center><h1>Data 245: Project</h1>
    <h2>NBA Prediction</h2>

<h3>Imports

In [1]:
import numpy as np, scipy as sc, pandas as pd, requests
import xml.etree.ElementTree as ET
from nba_api.stats.static import players
from nba_api.stats.endpoints import playercareerstats, playergamelog, playergamelogs, fantasywidget
from nba_api.stats.endpoints import infographicfanduelplayer, playerdashboardbylastngames, playernextngames
from nba_api.stats.endpoints import commonplayerinfo, commonallplayers
import json
import time
from IPython.display import Audio

<h3>Notes

Scoring Rules:
- point +1
- 3pt made +0.5
- rebound +1.25
- assist +1.5
- steal +2
- turnover -0.5
- 10x2 +1.5
- 10x3 +3

Gen restrictions:
- 8 players
- min 2 teams
- 1x5 basic positions, 1G, 1F, 1 Util

<h3>Sound (not neccessary for function)

In [2]:
'''
Audio Source:
https://mixkit.co/free-sound-effects/beep/
Positive Interface Beep
'''
sound_file = './sound/beep.wav'

<h3>Injuries Dataframe

In [3]:
'''
url = 'https://www.fantasybasketballnerd.com/service/injuries'
res = requests.get(url)
content = res.text
tree = ET.ElementTree(ET.fromstring(content))
team = ""
player = []
injuries = []
cols = ["name","injury","notes","updated"]
for elem in tree.iter():
    if elem.tag == "Team":
        team = elem.attrib['code']
    elif elem.tag == "Player":
        player.append(team)
    elif elem.tag in cols:
        player.append(elem.text)
    if elem.tag == "updated":
        injuries.append(player)
        player = []
injuries
cols.insert(0,"team")
injuries_df = pd.DataFrame.from_records(injuries,columns=cols)'''

'\nurl = \'https://www.fantasybasketballnerd.com/service/injuries\'\nres = requests.get(url)\ncontent = res.text\ntree = ET.ElementTree(ET.fromstring(content))\nteam = ""\nplayer = []\ninjuries = []\ncols = ["name","injury","notes","updated"]\nfor elem in tree.iter():\n    if elem.tag == "Team":\n        team = elem.attrib[\'code\']\n    elif elem.tag == "Player":\n        player.append(team)\n    elif elem.tag in cols:\n        player.append(elem.text)\n    if elem.tag == "updated":\n        injuries.append(player)\n        player = []\ninjuries\ncols.insert(0,"team")\ninjuries_df = pd.DataFrame.from_records(injuries,columns=cols)'

<h3>Get Active Players

In [4]:
x = commonallplayers.CommonAllPlayers().get_data_frames()[0]

In [5]:
active_players = x[x['TO_YEAR'] == '2020']

In [6]:
ap = list(active_players['PERSON_ID'])

In [7]:
len(ap)

536

<h3>Season Stats Data Frame Building

In [8]:
t_df = commonplayerinfo.CommonPlayerInfo(player_id=2544).get_data_frames()[0]
headers = list(t_df.columns.values)

In [9]:
df = pd.DataFrame(columns = headers)

In [10]:
e = 0
for player in ap:
    try:
        base = commonplayerinfo.CommonPlayerInfo(player_id=player, timeout=1000).get_data_frames()
        df = df.append(base[0])
        time.sleep(0.600)
    except:
        if e < 7:
            e+=1
            time.sleep(45)
            #print(e)
            base = commonplayerinfo.CommonPlayerInfo(player_id=player, timeout=1000).get_data_frames()
            df = df.append(base[0])
            continue
        else:
            print('break')
            break
Audio(sound_file, autoplay=True)

ValueError: rate must be specified when data is a numpy array or list of audio samples.

In [11]:
f_df = fantasywidget.FantasyWidget().get_data_frames()[0]

In [12]:
df.columns

Index(['PERSON_ID', 'FIRST_NAME', 'LAST_NAME', 'DISPLAY_FIRST_LAST',
       'DISPLAY_LAST_COMMA_FIRST', 'DISPLAY_FI_LAST', 'PLAYER_SLUG',
       'BIRTHDATE', 'SCHOOL', 'COUNTRY', 'LAST_AFFILIATION', 'HEIGHT',
       'WEIGHT', 'SEASON_EXP', 'JERSEY', 'POSITION', 'ROSTERSTATUS',
       'GAMES_PLAYED_CURRENT_SEASON_FLAG', 'TEAM_ID', 'TEAM_NAME',
       'TEAM_ABBREVIATION', 'TEAM_CODE', 'TEAM_CITY', 'PLAYERCODE',
       'FROM_YEAR', 'TO_YEAR', 'DLEAGUE_FLAG', 'NBA_FLAG', 'GAMES_PLAYED_FLAG',
       'DRAFT_YEAR', 'DRAFT_ROUND', 'DRAFT_NUMBER'],
      dtype='object')

In [13]:
f_df.columns

Index(['PLAYER_ID', 'PLAYER_NAME', 'PLAYER_POSITION', 'TEAM_ID',
       'TEAM_ABBREVIATION', 'GP', 'MIN', 'FAN_DUEL_PTS', 'NBA_FANTASY_PTS',
       'PTS', 'REB', 'AST', 'BLK', 'STL', 'TOV', 'FG3M', 'FGA', 'FG_PCT',
       'FTA', 'FT_PCT'],
      dtype='object')

In [14]:
df = df.rename(columns={"PERSON_ID": "PLAYER_ID"})

In [15]:
df = df.drop(['TEAM_ID', 'TEAM_ABBREVIATION'], axis = 1)

In [16]:
df =  df.join(f_df.set_index('PLAYER_ID'), on='PLAYER_ID')

In [17]:
keep = ['PLAYER_ID', 'FIRST_NAME', 'LAST_NAME',
       'BIRTHDATE', 'HEIGHT',
       'WEIGHT', 'SEASON_EXP', 'ROSTERSTATUS',
       'TEAM_ID', 'TEAM_NAME',
       'TEAM_ABBREVIATION', 'TEAM_CODE', 'TEAM_CITY',
       'DRAFT_YEAR', 'DRAFT_NUMBER', 'PLAYER_POSITION',
       'GP', 'MIN',
       'PTS', 'REB', 'AST', 'BLK', 'STL', 'TOV', 'FG3M', 'FGA', 'FG_PCT',
       'FTA', 'FT_PCT']

In [18]:
df = df.drop([i for i in df.columns if i not in keep], axis = 1)

In [19]:
df['DRAFT_NUMBER'].unique()

array(['20', 'Undrafted', '12', '14', '2', '17', '21', '22', '8', '30',
       '15', '60', '51', '3', '23', '43', '9', '1', '27', '40', '6', '7',
       '48', '25', '19', '38', '42', '36', '18', '35', None, '31', '44',
       '57', '39', '13', '28', '50', '47', '10', '45', '46', '33', '49',
       '32', '11', '4', '41', '5', '34', '29', '26', '55', '24', '59',
       '37', '16', '52', '54', '58', '56', '53'], dtype=object)

In [20]:
dic = {}
for i in ['GP', 'MIN', 'PTS', 'REB', 'AST', 'BLK', 'STL', 'TOV', 'FG3M', 'FGA', 'FG_PCT','FTA', 'FT_PCT']:
    dic[i] = 'CURRENT_SEASON_'+i    

In [21]:
df = df.rename(columns=dic)

In [22]:
df['DRAFT_NUMBER'] = df['DRAFT_NUMBER'].replace({'Undrafted':61, None:61})

In [23]:
df.columns

Index(['PLAYER_ID', 'FIRST_NAME', 'LAST_NAME', 'BIRTHDATE', 'HEIGHT', 'WEIGHT',
       'SEASON_EXP', 'ROSTERSTATUS', 'TEAM_NAME', 'TEAM_CODE', 'TEAM_CITY',
       'DRAFT_YEAR', 'DRAFT_NUMBER', 'PLAYER_POSITION', 'TEAM_ID',
       'TEAM_ABBREVIATION', 'CURRENT_SEASON_GP', 'CURRENT_SEASON_MIN',
       'CURRENT_SEASON_PTS', 'CURRENT_SEASON_REB', 'CURRENT_SEASON_AST',
       'CURRENT_SEASON_BLK', 'CURRENT_SEASON_STL', 'CURRENT_SEASON_TOV',
       'CURRENT_SEASON_FG3M', 'CURRENT_SEASON_FGA', 'CURRENT_SEASON_FG_PCT',
       'CURRENT_SEASON_FTA', 'CURRENT_SEASON_FT_PCT'],
      dtype='object')

<h4>Gamelog Dictionary

In [24]:
Game_Logs = {}

In [25]:
e = 0
for player in ap:
    try:
        base = playergamelog.PlayerGameLog(player_id=player, timeout=1000).get_data_frames()
        Game_Logs[player] = base[0]
        time.sleep(0.600)
    except:
        if e < 7:
            e+=1
            time.sleep(45)
            #print(e)
            base = playergamelog.PlayerGameLog(player_id=player, timeout=1000).get_data_frames()
            Game_Logs[player] = base[0]
            continue
        else:
            print('break')
            break
Audio(sound_file, autoplay=True)

ValueError: rate must be specified when data is a numpy array or list of audio samples.

In [26]:
for player in ap:
    if len(Game_Logs[player]) == 0:
        Game_Logs.pop(player)

In [27]:
def dkings_score(row):
    try:
        threes = row['FG3M']
        r = row['REB']
        a = row['AST']
        s = row['STL']
        b = row['BLK']
        t = row['TOV']
        p = row['PTS']
        score = 0.5*threes + 1.25*r + 1.5*a + 2*s +2*b + -0.5*t + p
        dd = sum(1 for i in [p, r, a, s, b] if i>=10)
        if dd == 2:
            score += 1.5
        if dd > 2:
            score += 4.5
        return(score)
    except:
        print(row)
        pass

In [28]:
to_be_averaged= ['MIN', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA',
       'OREB', 'DREB', 'AST', 'STL', 'BLK', 'TOV',
       'PTS']

In [40]:
# Add rest days into dataframe

def resting(select_df):
    
    # Transform string of game_date to datetime format

    select_df['GAME_DATETIME'] = pd.to_datetime(select_df['GAME_DATE'], format='%b %d, %Y')
    select_df['GAME_DATENEW'] = select_df['GAME_DATETIME'].dt.date
    
    # Take diff between each date

    select_df['GAME_DATENEW']=pd.to_datetime(select_df['GAME_DATENEW'])
    select_df['rest_days'] = select_df['GAME_DATENEW'].diff()
    
    # Rest days column is added

    select_df['rest_days'] = select_df['rest_days'].abs()
    select_df.drop(columns=['GAME_DATETIME', 'GAME_DATENEW'])

In [29]:
for frame in Game_Logs:
    x = Game_Logs[frame]
    x['DKING_SCORE'] = x.apply(lambda row: dkings_score(row), axis=1)
    x['TEAM'] = x.apply(lambda row: row['MATCHUP'][:3], axis=1)
    x['OPPOSING_TEAM'] = x.apply(lambda row: row['MATCHUP'][-3:], axis=1)
    x['HOME_AWAY'] = x.apply(lambda row: 1 if '@' in row['MATCHUP'] else 0, axis = 1)
    x.rename({'WL': 'PREVIOUS_WL'})
    for column in to_be_averaged:
        x['SEASON_'+ column] = x[column].rolling(window = len(x), min_periods = 1).mean()
        x['5_GAME_AVG_'+ column] = x[column].rolling(window = 5, min_periods = 1).mean()
    Game_Logs[frame] = x.drop(['MIN', 'MATCHUP', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'PLUS_MINUS', 'VIDEO_AVAILABLE'], axis = 1)

In [30]:
not_shift = ['SEASON_ID', 'Player_ID', 'Game_ID', 'GAME_DATE','DKING_SCORE',
       'TEAM', 'OPPOSING_TEAM', 'HOME_AWAY']

In [42]:
testing = Game_Logs[ap[0]]

In [43]:
to_shift = [i for i in testing.columns if i not in not_shift]

In [44]:
for frame in Game_Logs:
    for column in to_shift:
        Game_Logs[frame][column]= Game_Logs[frame][column].shift(1)
    Game_Logs[frame] = Game_Logs[frame][[c for c in Game_Logs[frame] if c != 'DKING_SCORE'] 
       + ['DKING_SCORE']]
    resting(Game_Logs[frame])

In [45]:
testing.head()

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,WL,TEAM,OPPOSING_TEAM,HOME_AWAY,SEASON_MIN,5_GAME_AVG_MIN,...,SEASON_BLK,5_GAME_AVG_BLK,SEASON_TOV,5_GAME_AVG_TOV,SEASON_PTS,5_GAME_AVG_PTS,DKING_SCORE,GAME_DATETIME,GAME_DATENEW,rest_days
0,22020,1630173,0022000888,"APR 21, 2021",NaN,MIA,SAS,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.25,NaT,NaT,NaT
1,22020,1630173,0022000871,"APR 19, 2021",NaN,MIA,HOU,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,22.25,2021-04-21,2021-04-21,NaT
2,22020,1630173,0022000852,"APR 16, 2021",NaN,MIA,MIN,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.25,2021-04-19,2021-04-19,2 days
3,22020,1630173,0022000838,"APR 14, 2021",W,MIA,DEN,1,2.0,2.0,...,0.0,0.0,0.0,0.0,1.0,1.0,10.00,2021-04-16,2021-04-16,3 days
4,22020,1630173,0022000827,"APR 13, 2021",W,MIA,PHX,1,15.5,15.5,...,0.0,0.0,1.5,1.5,4.5,4.5,6.50,2021-04-14,2021-04-14,2 days
